In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [29]:
df = pd.read_csv("ProjectTweets.csv")

In [30]:
df.head() # let's see what we are dealing with

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,1,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,2,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,3,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,4,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,5,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [31]:
# Create a new DataFrame with the first row containing the original column names
new_row = pd.DataFrame([df.columns], columns=df.columns)

# Concatenate the new row with the original DataFrame
df = pd.concat([new_row, df], ignore_index=True)
df.drop(columns=['0'], inplace=True)

df.reset_index(drop=True, inplace=True)

In [32]:
df.columns = ["tweet_id", "date", "flag", "user", "text"]
df.head()

,tweet_id,date,flag,user,text
0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 5 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   tweet_id  1600000 non-null  object
 1   date      1600000 non-null  object
 2   flag      1600000 non-null  object
 3   user      1600000 non-null  object
 4   text      1600000 non-null  object
dtypes: object(5)
memory usage: 61.0+ MB


In [62]:
# remove commas for csv split
df['text'] = df['text'].str.replace(',', '')

In [63]:
# testing
contains_char = df['text'].str.contains(',')
count = contains_char.sum()
print(f"Number of rows: {count}")

Number of rows: 0


In [64]:
# exporting csv to store on HBASE
df.to_csv("tweets.csv", header=False, index=False, sep=',')

### Loading data into MongoDB (NoSql)
Not working. Only to demostrate the evaluation process

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, TimestampType
from pyspark.sql.functions import to_timestamp

In [8]:
spark = SparkSession.builder.appName("MongoDB") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12-10.2.0") \
    .config("spark.mongodb.connection.uri", "mongodb://localhost:27017/") \
    .config("spark.mongodb.database", "twitter_db") \
    .config("spark.mongodb.collection", "tweets") \
    .getOrCreate()

23/10/22 12:46:25 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [9]:
spark.stop() #need to stop and restart to have a fresh new session

In [10]:
spark = SparkSession.builder.appName("MongoDB") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12-10.2.0") \
    .config("spark.mongodb.connection.uri", "mongodb://localhost:27017/") \
    .config("spark.mongodb.database", "twitter_db") \
    .config("spark.mongodb.collection", "tweets") \
    .getOrCreate()

In [11]:
print(spark.sparkContext.getConf().toDebugString())

spark.app.id=local-1697975211175
spark.app.name=MongoDB
spark.app.startTime=1697975210785
spark.app.submitTime=1697975158225
spark.driver.extraJavaOptions=-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false
spark.driver.host=10.0.2.15
spark.driver

In [12]:
print(f'The PySpark {spark.version} version is running...')

The PySpark 3.5.0 version is running...


In [13]:
my_schema = StructType([ StructField("n/a", StringType(), True)\
                   ,StructField("tweet_id", StringType(), True)\
                      ,StructField("date", StringType(), True)\
                      ,StructField("flag", StringType(), True)\
                      ,StructField("user", StringType(), True)\
                      ,StructField("text", StringType(), True)])

In [14]:
data = spark.createDataFrame(df, schema=my_schema)

In [13]:
data.show(5)

2023-10-19 15:27:24,448 WARN scheduler.TaskSetManager: Stage 0 contains a task of very large size (194599 KiB). The maximum recommended task size is 1000 KiB.


+---+----------+--------------------+--------+---------------+--------------------+
|n/a|  tweet_id|                date|    flag|           user|                text|
+---+----------+--------------------+--------+---------------+--------------------+
|  0|1467810369|Mon Apr 06 22:19:...|NO_QUERY|_TheSpecialOne_|@switchfoot http:...|
|  1|1467810672|Mon Apr 06 22:19:...|NO_QUERY|  scotthamilton|is upset that he ...|
|  2|1467810917|Mon Apr 06 22:19:...|NO_QUERY|       mattycus|@Kenichan I dived...|
|  3|1467811184|Mon Apr 06 22:19:...|NO_QUERY|        ElleCTF|my whole body fee...|
|  4|1467811193|Mon Apr 06 22:19:...|NO_QUERY|         Karoli|@nationwideclass ...|
+---+----------+--------------------+--------+---------------+--------------------+
only showing top 5 rows



In [15]:
data.write.format("mongodb").mode("append").save()

23/10/22 12:47:41 WARN TaskSetManager: Stage 0 contains a task of very large size (194602 KiB). The maximum recommended task size is 1000 KiB.
23/10/22 12:47:48 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 0 (TID 0): Attempting to kill Python Worker
23/10/22 12:47:48 ERROR Executor: Exception in task 0.0 in stage 0.0 (TID 0)
java.lang.NoSuchMethodError: org.apache.spark.sql.catalyst.encoders.RowEncoder$.apply(Lorg/apache/spark/sql/types/StructType;)Lorg/apache/spark/sql/catalyst/encoders/ExpressionEncoder;
	at com.mongodb.spark.sql.connector.schema.InternalRowToRowFunction.<init>(InternalRowToRowFunction.java:44)
	at com.mongodb.spark.sql.connector.schema.RowToBsonDocumentConverter.<init>(RowToBsonDocumentConverter.java:84)
	at com.mongodb.spark.sql.connector.write.MongoDataWriter.<init>(MongoDataWriter.java:74)
	at com.mongodb.spark.sql.connector.write.MongoDataWriterFactory.createWriter(MongoDataWriterFactory.java:53)
	at org.apache.spark.sql.execution.data

Py4JJavaError: An error occurred while calling o132.save.
: org.apache.spark.SparkException: Writing job failed.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.writingJobFailedError(QueryExecutionErrors.scala:902)
	at org.apache.spark.sql.execution.datasources.v2.V2TableWriteExec.writeWithV2(WriteToDataSourceV2Exec.scala:411)
	at org.apache.spark.sql.execution.datasources.v2.V2TableWriteExec.writeWithV2$(WriteToDataSourceV2Exec.scala:359)
	at org.apache.spark.sql.execution.datasources.v2.AppendDataExec.writeWithV2(WriteToDataSourceV2Exec.scala:225)
	at org.apache.spark.sql.execution.datasources.v2.V2ExistingTableWriteExec.run(WriteToDataSourceV2Exec.scala:337)
	at org.apache.spark.sql.execution.datasources.v2.V2ExistingTableWriteExec.run$(WriteToDataSourceV2Exec.scala:336)
	at org.apache.spark.sql.execution.datasources.v2.AppendDataExec.run(WriteToDataSourceV2Exec.scala:225)
	at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.result$lzycompute(V2CommandExec.scala:43)

### Data Processing with MapReduce/Spark
- Load data into Hbase from csv
- Run Map-Reduce jobs on the tweets data to get insights and pre process the data (ex: removing empty fields)
- Write the output file back to Hbase for further work

Please refer to the report and attached python scripts for additional details on those steps.

### Working with Post Processed Data
##### Option 1 - Reading data from Hbase

In [2]:
import happybase

In [3]:
# Connect to HBase
connection = happybase.Connection('localhost')
table = connection.table('processed_tweets')

In [4]:
%%time

# Read data from HBase
data = []
for key, value in table.scan():
    row = {'row_key': key.decode()}
    for column, val in value.items():
        column_name = column.decode().split(':')[1]
        row[column_name] = val.decode()
    data.append(row)

CPU times: user 45.1 s, sys: 1.2 s, total: 46.3 s
Wall time: 1min 14s


In [5]:
%%time
df_hb = pd.DataFrame(data)

CPU times: user 2.54 s, sys: 56 ms, total: 2.59 s
Wall time: 3.13 s


In [6]:
df_hb.head(3)

,row_key,date,text,user
0,1,Fri Apr 17 20:30:31 PDT 2009,Working songg aunt nan kinda hard crying I wri...,xoLovebug224
1,10,Fri Apr 17 20:30:47 PDT 2009,USEOFFORCEENT sorry loss honey I hope hurt hea...,customcanvasart
2,100,Fri Apr 17 20:32:16 PDT 2009,IAmCastiel Get rest\t,taintedmango


##### Option 2 - Reading data from Hadoop using Spark

In [11]:
from pyspark.sql import SparkSession

In [14]:
spark = SparkSession.builder.appName("Tweets").getOrCreate()

In [15]:
%%time
data_s = spark.read.text("/CA/processedTweets.csv")

CPU times: user 3.85 ms, sys: 308 µs, total: 4.16 ms
Wall time: 5.34 s


In [16]:
%%time
df_spark = data_s.toPandas()

CPU times: user 10.7 s, sys: 992 ms, total: 11.7 s
Wall time: 44.2 s


In [17]:
df_spark.head(3)

,value
0,"1,Fri Apr 17 20:30:31 PDT 2009,xoLovebug224,Wo..."
1,"2,Fri Apr 17 20:30:34 PDT 2009,Kerry_Baker,can..."
2,"3,Fri Apr 17 20:30:38 PDT 2009,glamorusindie81..."


#### Sentiment Analysis
Using *df_hb* and vader

In [7]:
import nltk
from nltk.tokenize import word_tokenize, RegexpTokenizer # tokenize words
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

nltk.download('vader_lexicon') # get lexicons data
nltk.download('punkt') # for tokenizer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/hduser/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /home/hduser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
# visualization
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = (10, 8) # default plot size
import seaborn as sns
sns.set(style='whitegrid', palette='colorblind')
from wordcloud import WordCloud

In [27]:
df_hb.to_csv('file.csv', index=False)

In [9]:
def analyze_sentiment(data):
    analysis = SIA().polarity_scores(data)
    if analysis['compound'] > 0:
        return 'Positive'
    elif analysis['compound'] == 0:
        return 'Neutral'
    else:
        return 'Negative'

In [25]:
df_hb['sentiment'] = df_hb['text'].apply(lambda x : analyze_sentiment(x))

KeyboardInterrupt: 

The previous command has been manually stopped due to never ending execution time, data is too big for this approach so spark rdd will be applied.

In [35]:
import gc
gc.collect()

592

In [36]:
spark.catalog.clearCache()